In [7]:
import pandas as pd
import csv
import os
import time
from collections import defaultdict
import numpy as np
import itertools
import db_structure
import db_structure_v2
import networkx as nx
from imp import reload
import matplotlib.pyplot as plt
import utilities as u
from flask import jsonify
import logging
# import exceptions
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

logging.basicConfig(format=' %(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
%load_ext autoreload
%aimport -web
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
draw_db = db

G = nx.DiGraph()
G.add_nodes_from(draw_db.table_names)
for table_name in draw_db.table_names:
    for sibling in draw_db.find_table_siblings(table_name):
        G.add_edge(table_name, sibling)
        G.add_edge(sibling, table_name)
    for child in draw_db.find_table_children(table_name):
        G.add_edge(table_name, child)
plt.figure(figsize=(10,10))

#draw with networkx built-ins
nx.draw_networkx(G, node_shape="None", width=0.2)

#OR

#draw using pygraphviz
#A = nx.nx_agraph.to_agraph(G)
#H = nx.nx_agraph.from_agraph(A)
#nx.draw_spring(H, node_shape="None", with_labels=True, width=0.2)

In [28]:
x = db_structure_v2.DBMaker(directory_path='datasets/TOPICC/')

 2019-12-16 15:16:26,874 - INFO - Loading datasets/TOPICC/
 2019-12-16 15:16:26,877 - INFO - Calculating metadata for files in directory datasets/TOPICC/
 2019-12-16 15:16:28,872 - DEBUG - Metadata: defaultdict(<class 'dict'>, {'CARDIACPROCEDURES': {'file': 'CARDIACPROCEDURES.csv', 'columns': [{'name': 'PudID', 'type': 'MANY'}, {'name': 'siteid', 'type': 'MANY'}, {'name': 'Occurrence', 'type': 'MANY'}, {'name': 'FormStatus', 'type': 'MANY'}, {'name': 'ItemGroupRepeatKey', 'type': 'MANY'}, {'name': 'PrimaryProc1', 'type': 'MANY'}, {'name': 'PrimaryProc2', 'type': 'MANY'}, {'name': 'PrimProcOt', 'type': 'ONE'}, {'name': 'SecondProcOt', 'type': 'ONE'}, {'name': 'EVENTSTARTDAY', 'type': 'MANY'}, {'name': 'SecondProc11', 'type': 'MANY'}, {'name': 'SecondProc12', 'type': 'MANY'}, {'name': 'SecondProc13', 'type': 'MANY'}, {'name': 'SecondProc14', 'type': 'MANY'}, {'name': 'SecondProc15', 'type': 'MANY'}, {'name': 'SecondProc16', 'type': 'MANY'}, {'name': 'SecondProc17', 'type': 'MANY'}, {'nam

In [29]:
x.add_global_fk('PudID')

 2019-12-16 15:16:30,437 - INFO - Column PudID is in more than 2 tables. Will create a new joining table. Found in ['CARDIACPROCEDURES', 'CARDIACSURGERY', 'CAREPROCESSES', 'CATASTROPHICEVENTS', 'CPR', 'DEATH', 'HOSPITALADMIT', 'HOSPITALADMIT_CE', 'HOSPITALDISCHARGE', 'LIMITOFCARE', 'PHYSIOSTATUS', 'PICUADMIT', 'PICUADMIT_ADMITSECONDDX', 'PICUADMIT_AHD', 'PICUADMIT_CHD', 'PICUADMIT_CHRONICDX', 'PICUDISCHARGE', 'PICUDISCHARGE_DISCHRONICDX', 'PICUDISCHARGE_DISSECONDDX', 'SURGERY']


['CARDIACPROCEDURES',
 'CARDIACSURGERY',
 'CAREPROCESSES',
 'CATASTROPHICEVENTS',
 'CPR',
 'DEATH',
 'HOSPITALADMIT',
 'HOSPITALADMIT_CE',
 'HOSPITALDISCHARGE',
 'LIMITOFCARE',
 'PHYSIOSTATUS',
 'PICUADMIT',
 'PICUADMIT_ADMITSECONDDX',
 'PICUADMIT_AHD',
 'PICUADMIT_CHD',
 'PICUADMIT_CHRONICDX',
 'PICUDISCHARGE',
 'PICUDISCHARGE_DISCHRONICDX',
 'PICUDISCHARGE_DISSECONDDX',
 'SURGERY']

In [30]:
x.metadata

defaultdict(dict,
            {'CARDIACPROCEDURES': {'file': 'CARDIACPROCEDURES.csv',
              'columns': [{'name': 'PudID', 'type': 'MANY'},
               {'name': 'siteid', 'type': 'MANY'},
               {'name': 'Occurrence', 'type': 'MANY'},
               {'name': 'FormStatus', 'type': 'MANY'},
               {'name': 'ItemGroupRepeatKey', 'type': 'MANY'},
               {'name': 'PrimaryProc1', 'type': 'MANY'},
               {'name': 'PrimaryProc2', 'type': 'MANY'},
               {'name': 'PrimProcOt', 'type': 'ONE'},
               {'name': 'SecondProcOt', 'type': 'ONE'},
               {'name': 'EVENTSTARTDAY', 'type': 'MANY'},
               {'name': 'SecondProc11', 'type': 'MANY'},
               {'name': 'SecondProc12', 'type': 'MANY'},
               {'name': 'SecondProc13', 'type': 'MANY'},
               {'name': 'SecondProc14', 'type': 'MANY'},
               {'name': 'SecondProc15', 'type': 'MANY'},
               {'name': 'SecondProc16', 'type': 'MANY'},
     